In [55]:
import numpy as np
import sys
from numpy.core.fromnumeric import shape, size
from random import random, randint
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [48]:
randint(-1000,1)

-709

In [56]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def Loss(Y, Y_predicted):
    return(sum(np.square(Y - Y_predicted))/Y.size).sum()

def rand(row, column):
  returned = np.empty((row, column))
  for j in range(row):
      rando=[]
      for i in range(column):
          rando.append(random())
          i+=1
      returned[j] = rando
  return returned


def Normalize(X):
    return((X-X.min())/(X.max()-X.min()))

def Binarize(X):
    X [X < 100] = 0
    X [X >= 100] = 1       
    return X

def calculate_accuracy(predicted_y, y):
    guessWrong = 0
    if(len(predicted_y) == len(y)):
        ind = 0
        for i in predicted_y:
            if(i != y[ind]):
                guessWrong = guessWrong + 1
            ind+=1
        acc = ( (len(predicted_y)- guessWrong) / len(predicted_y))
        return (acc*100)
    return 0

In [57]:
class Layer:
    nerweight =0
    inputs = 0
    outputs=[]
    delta=0

    def __init__(self, neuronNumber,length):
        self.nerweight =rand(neuronNumber ,length)
        #self.bais = rand(neuronNumber , 1)

    def feedForward(self, inputs):
        self.inputs = inputs
        self.outputs  = sigmoid(np.dot(np.array(self.nerweight), np.array(self.inputs)))
        return self.outputs 

    
    def fit(self, eta, target, outj, sumDelta):
        if(size(sumDelta)==0):
            self.delta=(target-self.outputs)*self.outputs*(1-self.outputs)
            dw = eta*np.transpose([self.delta])*np.transpose(outj)
            self.nerweight = self.nerweight + dw 
            return np.dot(self.delta, self.nerweight)
        else:
            self.delta=sumDelta*self.outputs*(1-self.outputs)
            dw = eta*self.delta*np.transpose([outj])
            self.nerweight = self.nerweight + dw.reshape(len(self.nerweight), len(dw))
            return np.dot(self.delta, self.nerweight)
            

In [4]:
def getMax(ma):
    for j in range(shape(ma)[0]):
        cu = ma[j]
        ar=np.where(cu == np.amax(cu))
        for i in range (len(cu)):
            if(i == ar[0]):
                cu[i]=1
            else:
                cu[i]=0
        ma[j]= cu
    return ma

In [5]:
def accuracy_here(predicted_y, y):
    guess_wrong=0
    for i in range(len(predicted_y)):
        if(not np.array_equiv(predicted_y[i],y[i])):
            guess_wrong+=1
    acc = ((len(predicted_y)- guess_wrong) / len(predicted_y))
    return (acc*100)

In [145]:
class Network:
    layers = 0
    layernum=0
    length=0
    orin=0
    inputs =0
    target =0
    def __init__(self, NumNeuron, inputs, target):
        self.layers = []
        self.orin = inputs
        self.inputs =inputs
        self.target = target
        length = shape(inputs)[1]
        for i in range (len(NumNeuron)):
            layer = Layer(NumNeuron[i], length)  
            length = NumNeuron[i]   
            self.layers.append(layer)

    def feedForward(self, datafeeded, data):
        current = data[datafeeded]
        for i in range(len(self.layers)):
            current= self.layers[i].feedForward(current)
        return current
    
    def fit(self, eta, leranRate):
        for j in range(leranRate):
            now = np.empty((shape(self.target)))
            for k in range((shape(self.orin)[0])):
                now[k] =self.feedForward(k,self.inputs )
                sumdelta = []
                for i in range(len(self.layers)-1, 0, -1):
                    sumdelta =self.layers[i].fit(eta, self.target[k], self.layers[i-1].outputs, sumdelta)
                sumdelta =self.layers[0].fit(eta, self.target[k], self.orin[k], sumdelta)
                now[k] =self.feedForward(k, self.inputs )
            print (accuracy_here(getMax(now), train_y))


In [7]:
def centroid(X):
    M00=np.sum(X.sum(axis=1))
    M10 =np.array([])
    M01 =np.array([])
    for i in range(X.shape[0]):
        M10= np.append(M10, (X[i].sum(axis=0)*(i+1)))
        
    for i in range(X.shape[1]):
        M01 = np.append(M01,(X[:,i].sum(axis=0)*(i+1)))
    M10 = np.sum(M10)
    M01 = np.sum(M01)
    if(M00==0):
        Xc= 0
        Yc= 0
    else:
        Xc = M10/M00
        Yc = M01/M00
    return np.array([Xc, Yc])

In [8]:
def FeatureImage(X):
    vector=np.array([])
    for i in range(2):
        for j in range(4):              #0-7, 0--14  
            vector= np.append(vector, centroid(X[j*7:((j+1)*7) ,i*14:((i+1)*14)])) 
    return vector

In [9]:
def plot_digit(X, y, idx):
    plt.imshow(X[idx], cmap='Greys', interpolation='nearest')
    plt.title('true label: %d' % y[idx])
    plt.show()

In [10]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X.shape

(60000, 28, 28)

In [11]:
train_X =Binarize(train_X)
test_X = Binarize(test_X)

In [12]:
vec = np.array([])
for i in range(60000):
    vec = np.append(vec, FeatureImage(train_X[i]))
vec.shape

(960000,)

In [13]:
vec =vec.reshape(60000, 16)
vec.shape

(60000, 16)

In [14]:
vecTest = np.array([])
for i in range(10000):
    vecTest = np.append(vecTest, FeatureImage(test_X[i]))
print(vecTest.shape)
vecTest =vecTest.reshape(10000, 16)
print(vecTest.shape)

(160000,)
(10000, 16)


In [15]:
def fun (num):
    n = np.empty((len(num), 10))
    for i in range (len(num)):
        re = np.zeros((1 , 10))
        re[:,num[i]] = 1
        n[i] =re
    return n

In [16]:
test_y = fun(test_y)


In [17]:
train_y =fun (train_y)

In [18]:
vec[0]

array([ 7.        , 13.        ,  3.        , 12.12      ,  6.75      ,
       13.25      ,  2.69230769,  9.96153846,  6.5625    ,  5.6875    ,
        3.        ,  2.5       ,  4.20689655,  3.62068966,  1.        ,
        1.5       ])

In [174]:
mis3 = Network([2,3, 10], vec[0:60000], train_y[0:60000])

In [175]:
mis3.fit(5, 5)


52.00666666666667
52.01
52.01
52.01
52.01


In [170]:
preY = np.empty((shape(test_y)))

In [171]:
for i in range(len(test_y)):
    preY[i]=mis3.feedForward(i, vecTest)


In [172]:
print (accuracy_here(getMax(preY), test_y))

10.100000000000001


In [123]:
one = Network([2, 3,10], vec[0:1000], train_y[0:1000])

In [126]:
one.fit(2, 10000)

32.7
33.0


KeyboardInterrupt: 

In [ ]:
def accuracy_here(predicted_y, y):
    guess_wrong=0
    for i in range(len(predicted_y)):
        if(not np.array_equiv(predicted_y[i],y[i])):
            guess_wrong+=1
    acc = ((len(predicted_y)- guess_wrong) / len(predicted_y))
    return (acc*100)

11.0

In [ ]:
iris = load_iris() # Sepal Length, Sepal Width, Petal Length and Petal Width
X = iris.data
Y = iris.target
print(Y)
#My split function
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [ ]:
IRNet = Network([2,2], X_train, Y_train)